In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

# path to json file that stores MFCCs and genre labels for each processed segment
DATA_PATH = "mfcc_data_20s.json"

def load_data(data_path):
    """Loads training dataset from json file.

        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    # convert lists to numpy arrays
    X = np.array(data["mfccs"])
    y = np.array(data["labels"])
    genres = data["genres"]

    print("Data succesfully loaded!")

    return  X, y, genres

In [2]:
# load data
X, y, genres = load_data(DATA_PATH)

Data succesfully loaded!


In [3]:
# create train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [4]:
# build network topology
model = keras.Sequential([

    # input layer
    keras.layers.Flatten(input_shape=(X.shape[1], X.shape[2])),

    # 1st dense layer
    keras.layers.Dense(512, activation='relu'),

    # 2nd dense layer
    keras.layers.Dense(256, activation='relu'),

    # 3rd dense layer
    keras.layers.Dense(64, activation='relu'),

    # output layer
    keras.layers.Dense(len(genres), activation='softmax')
])

# compile model
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 17240)             0         
                                                                 
 dense (Dense)               (None, 512)               8827392   
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 64)                16448     
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 8,975,818
Trainable params: 8,975,818
Non-trainable params: 0
_________________________________________________________________


In [5]:
# train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=50)

Epoch 1/50
22/22 [==============================] - 3s 95ms/step - loss: 34.6969 - accuracy: 0.2275 - val_loss: 15.9691 - val_accuracy: 0.2600
Epoch 2/50
22/22 [==============================] - 2s 86ms/step - loss: 9.6708 - accuracy: 0.4192 - val_loss: 9.3555 - val_accuracy: 0.3633
Epoch 3/50
22/22 [==============================] - 2s 86ms/step - loss: 4.3829 - accuracy: 0.5737 - val_loss: 7.4022 - val_accuracy: 0.3900
Epoch 4/50
22/22 [==============================] - 2s 86ms/step - loss: 2.3482 - accuracy: 0.7353 - val_loss: 9.1727 - val_accuracy: 0.4233
Epoch 5/50
22/22 [==============================] - 2s 86ms/step - loss: 1.6166 - accuracy: 0.7811 - val_loss: 7.8180 - val_accuracy: 0.3933
Epoch 6/50
22/22 [==============================] - 2s 90ms/step - loss: 0.8820 - accuracy: 0.8383 - val_loss: 8.8597 - val_accuracy: 0.3700
Epoch 7/50
22/22 [==============================] - 2s 92ms/step - loss: 0.6479 - accuracy: 0.8770 - val_loss: 8.1465 - val_accuracy: 0.4200
Epoch 8/50


In [6]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=64)
print("test loss, test acc:", results)

Evaluate on test data
5/5 [==============================] - 0s 14ms/step - loss: 8.3565 - accuracy: 0.4933
test loss, test acc: [8.356498718261719, 0.4933333396911621]


In [7]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for tests")
predictions = model.predict(X_test)

Generate predictions for tests
10/10 [==============================] - 0s 7ms/step


In [8]:
correct = [0 for genre in genres]
pred_data = [[0 for g in genres] for genre in genres]
genre_count = [0 for genre in genres]
for i in range(predictions.shape[0]):
    max_confidence = max(predictions[i])
    pred = np.argmax(predictions[i])
    actual = y_test[i]
    
    genre_count[actual] += 1
    pred_data[actual][pred] += 1
    
    if pred == actual:
        correct[actual] += 1
    print('Prediction:', genres[pred], '\tActual:', genres[actual], '\nConfidence:', max(predictions[i]),'\n')

Prediction: blues 	Actual: country 
Confidence: 1.0 

Prediction: blues 	Actual: blues 
Confidence: 0.9997887 

Prediction: country 	Actual: pop 
Confidence: 0.7209512 

Prediction: disco 	Actual: country 
Confidence: 0.7996491 

Prediction: hiphop 	Actual: blues 
Confidence: 0.9999938 

Prediction: rock 	Actual: rock 
Confidence: 0.9988218 

Prediction: jazz 	Actual: blues 
Confidence: 0.99999964 

Prediction: metal 	Actual: metal 
Confidence: 1.0 

Prediction: pop 	Actual: pop 
Confidence: 0.89061224 

Prediction: jazz 	Actual: blues 
Confidence: 0.9874442 

Prediction: hiphop 	Actual: country 
Confidence: 0.5500751 

Prediction: blues 	Actual: blues 
Confidence: 0.9993382 

Prediction: blues 	Actual: blues 
Confidence: 0.99681926 

Prediction: disco 	Actual: rock 
Confidence: 0.99975616 

Prediction: blues 	Actual: blues 
Confidence: 0.9783131 

Prediction: blues 	Actual: rock 
Confidence: 0.45514357 

Prediction: jazz 	Actual: jazz 
Confidence: 1.0 

Prediction: classical 	Actual: 

In [9]:
curr_genre = 0
for c,g in zip(correct,genre_count):
    print(genres[curr_genre], ': ', c / g * 100,'% correct')
    print('Prediction distribution: ', pred_data[curr_genre],'\n')
    curr_genre += 1

print('\nTotal: ', sum(correct) / sum(genre_count) * 100, '% correct')

blues :  41.37931034482759 % correct
Prediction distribution:  [12, 2, 0, 0, 2, 4, 3, 0, 1, 5] 

classical :  91.30434782608695 % correct
Prediction distribution:  [0, 21, 2, 0, 0, 0, 0, 0, 0, 0] 

country :  25.0 % correct
Prediction distribution:  [7, 0, 7, 5, 4, 1, 0, 0, 1, 3] 

disco :  25.0 % correct
Prediction distribution:  [5, 1, 1, 7, 0, 2, 4, 3, 2, 3] 

hiphop :  32.142857142857146 % correct
Prediction distribution:  [2, 0, 1, 3, 9, 3, 4, 1, 5, 0] 

jazz :  72.22222222222221 % correct
Prediction distribution:  [1, 4, 1, 2, 1, 26, 0, 0, 1, 0] 

metal :  87.87878787878788 % correct
Prediction distribution:  [1, 0, 0, 0, 1, 0, 29, 0, 1, 1] 

pop :  57.14285714285714 % correct
Prediction distribution:  [1, 0, 2, 1, 2, 1, 0, 16, 1, 4] 

reggae :  30.303030303030305 % correct
Prediction distribution:  [2, 2, 3, 2, 6, 7, 1, 0, 10, 0] 

rock :  32.35294117647059 % correct
Prediction distribution:  [10, 0, 1, 1, 0, 2, 5, 2, 2, 11] 


Total:  49.333333333333336 % correct
